<h1>Preprocess and export the data to file (Obsolete)</h1>

In [2]:
import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math
from random import shuffle

IMG_SIZE_PX = 50
#SLICE_COUNT = 20
SLICE_COUNT = 10

def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]


def mean(a):
    return sum(a) / len(a)


def process_data(patient,labels_df,img_px_size=50, hm_slices=20, visualize=False):
    
    label = labels_df.get_value(patient, 'cancer')
    path = data_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

    new_slices = []
    slices = [cv2.resize(np.array(each_slice.pixel_array),(img_px_size,img_px_size)) for each_slice in slices]
    
    chunk_sizes = math.ceil(len(slices) / hm_slices)
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    if len(new_slices) == hm_slices-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices+2:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
        
    if len(new_slices) == hm_slices+1:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    if visualize:
        fig = plt.figure()
        for num,each_slice in enumerate(new_slices):
            y = fig.add_subplot(4,5,num+1)
            y.imshow(each_slice, cmap='gray')
        plt.show()

    if label == 1: label=np.array([0,1])
    elif label == 0: label=np.array([1,0])
        
    return np.array(new_slices),label

#                                               stage 1 for real.
data_dir = 'D:/stage1/'
patients = os.listdir(data_dir)
labels = pd.read_csv('data/stage1_labels.csv', index_col=0)

much_data = []
for num,patient in enumerate(patients):
    if num % 100 == 0:
        print(num)
    try:
        img_data,label = process_data(patient,labels,img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT)
        #print(img_data.shape,label)
        much_data.append([img_data,label])
    except KeyError as e:
        print('This is unlabeled data!')

np.save('muchdata-{}-{}-{}.npy'.format(IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT), much_data)

0
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
100
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
200
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
300
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
400
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data!
This is unlabeled data

<h1>Load preprocessed data, train and save model (Obsolete)</h1>

In [2]:
import tensorflow as tf
import numpy as np

IMG_SIZE_PX = 50
SLICE_COUNT = 10

n_classes = 2
batch_size = 10

x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.8

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32]), name='w_conv1'),
               #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64]), name='w_conv2'),
               #                                  64 features
               'W_fc':tf.Variable(tf.random_normal([32448,1024]), name='w_fc'), #54080 = ceil(50/2/2) * ceil(50/2/2) * ceil(10/2/2) * 64
               #'W_fc':tf.Variable(tf.random_normal([54080,1024]), name='W_fc'), #54080 = ceil(50/2/2) * ceil(50/2/2) * ceil(20/2/2) * 64
               'out':tf.Variable(tf.random_normal([1024, n_classes]), name='w_out')}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32]), name='b_conv1'),
               'b_conv2':tf.Variable(tf.random_normal([64]), name='b_conv2'),
               'b_fc':tf.Variable(tf.random_normal([1024]), name='b_fc'),
               'out':tf.Variable(tf.random_normal([n_classes]), name='b_out')}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1,32448])
    #fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

much_data = np.load('muchdata-50-50-10.npy')
# If you are working with the basic sample data, use maybe 2 instead of 100 here... you don't have enough data to really do this
train_data = much_data[:-100]
validation_data = much_data[-10:]
del much_data

def train_neural_network(x):  
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        successful_runs = 0
        total_runs = 0
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for data in train_data:
                total_runs += 1
                try:
                    X = data[0]
                    Y = data[1]
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_runs += 1
                except Exception as e:
                    # I am passing for the sake of notebook space, but we are getting 1 shaping issue from one 
                    # input tensor. Not sure why, will have to look into it. Guessing it's
                    # one of the depths that doesn't come to 20.
                    pass
                    #print(str(e))
            
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
            
        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        
        print('fitment percent:',successful_runs/total_runs)
        
        saver = tf.train.Saver()
        saver.save(sess, 'my-save-dir/my-model-10')

# Run this locally:
train_neural_network(x)


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1 completed out of 10 loss: 147567120837.0
Accuracy: 0.7
Epoch 2 completed out of 10 loss: 16274273583.0
Accuracy: 0.7
Epoch 3 completed out of 10 loss: 5269966279.02
Accuracy: 0.6
Epoch 4 completed out of 10 loss: 2688508502.5
Accuracy: 0.5
Epoch 5 completed out of 10 loss: 1715656093.11
Accuracy: 0.6
Epoch 6 completed out of 10 loss: 925859933.625
Accuracy: 0.6
Epoch 7 completed out of 10 loss: 572327315.552
Accuracy: 0.5
Epoch 8 completed out of 10 loss: 273143603.493
Accuracy: 0.7
Epoch 9 completed out of 10 loss: 124091884.593
Accuracy: 0.5
Epoch 10 completed out of 10 loss: 108039876.408
Accuracy: 0.6
Done. Finishing accuracy:
Accuracy: 0.8
fitment percent: 1.0


<h1>Train and save model</h1>

In [19]:
import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math
import tensorflow as tf
from random import shuffle
import time

start_time = time.time() #start timer

IMG_SIZE_PX = 50
SLICE_COUNT = 20
#SLICE_COUNT = 10

cache_dir = "cache/"
data_dir = 'D:/stage1/'
model_save_folder = 'my-save-dir'
patients = os.listdir(data_dir)
shuffle(patients)
label_file = 'data/stage1_labels.csv'
labels = pd.read_csv(label_file, index_col=0)

n_classes = 2
batch_size = 300
training_size = 1000
validation_size = 100

keep_rate = 0.8

def clear_cache():
    folder = cache_dir[:-1]
    if os.path.isdir(folder):
        for the_file in os.listdir(folder):
            file_path = os.path.join(folder, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                #elif os.path.isdir(file_path): shutil.rmtree(file_path)
            except Exception as e:
                print(e)
    else:
        os.makedirs(folder)

def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]


def mean(a):
    return sum(a) / len(a)


def process_data(patient,labels_df,img_px_size=50, hm_slices=20):
    label = labels_df.get_value(patient, 'cancer')
    path = data_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

    new_slices = []
    slices = [cv2.resize(np.array(each_slice.pixel_array),(img_px_size,img_px_size)) for each_slice in slices]

    chunk_sizes = math.ceil(len(slices) / hm_slices)
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    while len(new_slices) < hm_slices:
        new_slices.append(new_slices[-1])

    while len(new_slices) > hm_slices:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    if label == 1: label=np.array([0,1])
    elif label == 0: label=np.array([1,0])

    return np.array(new_slices),label

def GetPatientsPreprocessedData(start, end):
    cacheFilePath = cache_dir + str(start) + '-' + str(end) + '.npy'
    if os.path.exists(cacheFilePath):
        X, Y = np.load(cacheFilePath)
        return list(X), list(Y)

    X = []
    Y = []

    for i in range(start, end):
        patient = patients[i]
        try:
            img_data,label = process_data(patient,labels,img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT)
            #print(img_data.shape,label)
            X.append(img_data)
            Y.append(label)
        except KeyError as e:
            #print('This is unlabeled data!')
            pass

    np.save(cacheFilePath, [X, Y])
    return [X, Y]

x = tf.placeholder('float')
y = tf.placeholder('float')

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #                                  64 features
               #'W_fc':tf.Variable(tf.random_normal([32448,1024])), #54080 = ceil(50/2/2) * ceil(50/2/2) * ceil(10/2/2) * 64
               'W_fc':tf.Variable(tf.random_normal([54080,1024]), name='W_fc'), #54080 = ceil(50/2/2) * ceil(50/2/2) * ceil(20/2/2) * 64
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    #fc = tf.reshape(conv2,[-1,32448])
    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.add(tf.matmul(fc, weights['out']), biases['out'])

    return output

clear_cache()
validation_data = GetPatientsPreprocessedData(len(patients) - validation_size, len(patients))

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)

    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        successful_runs = 0
        total_runs = 0

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(0, math.ceil(training_size / batch_size)): #train in batches
                print('Batch', i+1)
                total_runs += 1
                try:
                    X, Y = GetPatientsPreprocessedData(i * batch_size, min((i +
                        1) * batch_size, training_size))
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_runs += 1
                except Exception as e:
                    # I am passing for the sake of notebook space, but we are getting 1 shaping issue from one
                    # input tensor. Not sure why, will have to look into it. Guessing it's
                    # one of the depths that doesn't come to 20.
                    print(str(e))
                    pass

            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            print('Accuracy:',accuracy.eval({x:[i for i in validation_data[0]], y:[i for i in validation_data[1]]}))

        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i for i in validation_data[0]], y:[i for i in validation_data[1]]}))

        print('fitment percent:',successful_runs/total_runs)

        tf.add_to_collection('input', x)
        tf.add_to_collection('output', prediction)
        saver = tf.train.Saver()

        if not os.path.isdir(model_save_folder):
            os.makedirs(model_save_folder)

        saver.save(sess, model_save_folder + '/my-model')

# Run this locally:
train_neural_network(x)
durationS = time.time() - start_time
durationH = int(durationS / 3600)
durationS = durationS % 3600
durationM = int(durationS / 60)
durationS = durationS % 60
print("--- %s hours %s minutes %s seconds ---" % (durationH, durationM, durationS)) #print time taken

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/stage1/'

<h1>Same as above but this time there is additional pre-processing added</h1>

In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
import scipy.ndimage
from random import shuffle
import matplotlib.pyplot as plt
import cv2
import math
import tensorflow as tf
import time

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection 

start_time = time.time() #start timer

IMG_SIZE_PX = 50
SLICE_COUNT = 20

cache_dir = "cache/"
data_dir = 'D:/stage1/'
model_save_folder = 'my-save-dir'
patients = os.listdir(data_dir)
shuffle(patients)
label_file = 'data/stage1_labels.csv'
labels = pd.read_csv(label_file, index_col=0)

n_classes = 2
batch_size = 300
training_size = 1000
validation_size = 100
keep_rate = 0.8

# Load the scans in given folder path
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + scan[0].PixelSpacing, dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image
    
def normalize(image):
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

def zero_center(image):
    PIXEL_MEAN = 0.25
    image = image - PIXEL_MEAN
    return image

def firstPreprocess(patient):
    patient_img = load_scan(data_dir + patient)
    patient_pixels = get_pixels_hu(patient_img)
    pix_resampled, spacing = resample(patient_pixels, patient_img, [1,1,1])
    #segmented_lungs = segment_lung_mask(pix_resampled, False)
    segmented_lungs_fill = segment_lung_mask(pix_resampled, True)
    finalOutput = np.multiply(segmented_lungs_fill, pix_resampled)
    finalOutput = normalize(finalOutput)
    finalOutput = zero_center(finalOutput)
    return finalOutput

def clear_cache():
    folder = cache_dir[:-1]
    if os.path.isdir(folder):
        for the_file in os.listdir(folder):
            file_path = os.path.join(folder, the_file)
            try:
                if os.path.isfile(file_path):
                    os.unlink(file_path)
                #elif os.path.isdir(file_path): shutil.rmtree(file_path)
            except Exception as e:
                print(e)
    else:
        os.makedirs(folder)

def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def mean(a):
    return sum(a) / len(a)

def secondpreprocess(slices, img_px_size, hm_slices):
    new_slices = []
    slices = [cv2.resize(np.array(each_slice),(img_px_size,img_px_size)) for each_slice in slices]

    chunk_sizes = math.ceil(len(slices) / hm_slices)
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    while len(new_slices) < hm_slices:
        new_slices.append(new_slices[-1])

    while len(new_slices) > hm_slices:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    return np.array(new_slices)

def process_data(patient,labels_df,img_px_size=50, hm_slices=20):
    data = secondpreprocess(firstPreprocess(patient), img_px_size, hm_slices)
    label = labels_df.get_value(patient, 'cancer')

    if label == 1: label=np.array([0,1])
    elif label == 0: label=np.array([1,0])

    return data, label

def GetPatientsPreprocessedData(start, end):
    cacheFilePath = cache_dir + str(start) + '-' + str(end) + '.npy'
    if os.path.exists(cacheFilePath):
        X, Y = np.load(cacheFilePath)
        return list(X), list(Y)

    X = []
    Y = []

    for i in range(start, end):
        patient = patients[i]
        try:
            img_data,label = process_data(patient,labels,img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT)
            #print(img_data.shape,label)
            X.append(img_data)
            Y.append(label)
        except KeyError as e:
            #print('This is unlabeled data!')
            pass

    np.save(cacheFilePath, [X, Y])
    return [X, Y]

x = tf.placeholder('float')
y = tf.placeholder('float')

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #                                  64 features
               #'W_fc':tf.Variable(tf.random_normal([32448,1024])), #54080 = ceil(50/2/2) * ceil(50/2/2) * ceil(10/2/2) * 64
               'W_fc':tf.Variable(tf.random_normal([54080,1024]), name='W_fc'), #54080 = ceil(50/2/2) * ceil(50/2/2) * ceil(20/2/2) * 64
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)

    #fc = tf.reshape(conv2,[-1,32448])
    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.add(tf.matmul(fc, weights['out']), biases['out'])

    return output

clear_cache()
validation_data = GetPatientsPreprocessedData(len(patients) - validation_size, len(patients))

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)

    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        successful_runs = 0
        total_runs = 0

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(0, math.ceil(training_size / batch_size)): #train in batches
                print('Batch', i+1)
                total_runs += 1
                try:
                    X, Y = GetPatientsPreprocessedData(i * batch_size, min((i +
                        1) * batch_size, training_size))
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_runs += 1
                except Exception as e:
                    # I am passing for the sake of notebook space, but we are getting 1 shaping issue from one
                    # input tensor. Not sure why, will have to look into it. Guessing it's
                    # one of the depths that doesn't come to 20.
                    print(str(e))
                    pass

            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            print('Accuracy:',accuracy.eval({x:[i for i in validation_data[0]], y:[i for i in validation_data[1]]}))

        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i for i in validation_data[0]], y:[i for i in validation_data[1]]}))

        print('fitment percent:',successful_runs/total_runs)

        tf.add_to_collection('input', x)
        tf.add_to_collection('output', prediction)
        saver = tf.train.Saver()

        if not os.path.isdir(model_save_folder):
            os.makedirs(model_save_folder)

        saver.save(sess, model_save_folder + '/my-model')

# Run this locally:
train_neural_network(x)
durationS = time.time() - start_time
durationH = int(durationS / 3600)
durationS = durationS % 3600
durationM = int(durationS / 60)
durationS = durationS % 60
print("--- %s hours %s minutes %s seconds ---" % (durationH, durationM, durationS)) #print time taken

<class 'numpy.ndarray'> 324 346 346


<h1>Restore trained model and predict</h1>

In [2]:
import tensorflow as tf
import numpy as np
import csv
import dicom
import os
import cv2
import math

IMG_SIZE_PX = 50
SLICE_COUNT = 20
data_dir = 'D:/stage1/'
model_save_folder = 'my-save-dir'

def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]


def mean(a):
    return sum(a) / len(a)

def PrintPrediction(patients, prediction, file):
    with open(file, 'w') as f:
        writer = csv.writer(f)
        for i in range(len(patients)):
            writer.writerow([patients[i], '0' if prediction[i][1] < 0 else
                str(prediction[i][1])])

def process_data(patient,img_px_size=50, hm_slices=20):
    path = data_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

    new_slices = []
    slices = [cv2.resize(np.array(each_slice.pixel_array),(img_px_size,img_px_size)) for each_slice in slices]
    
    chunk_sizes = math.ceil(len(slices) / hm_slices)
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    if len(new_slices) == hm_slices-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices+2:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
        
    if len(new_slices) == hm_slices+1:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
        
    return np.array(new_slices)

with tf.Session() as sess:
    model_saver = tf.train.import_meta_graph(model_save_folder + '/my-model.meta')
    model_saver.restore(sess, model_save_folder + '/my-model')   
    x = tf.get_collection("input")[0]
    output = tf.get_collection("output")[0]
    print("Model restored.") 
    print('Initialized')
    #print(sess.run(tf.get_default_graph().get_tensor_by_name('w_conv1:0')))
    
    #collect list of preprocessed data on submission set
    inputData = []
    patients = []
    isColumnHeader = True

    with open('stage1_sample_submission.csv') as f:
        reader = csv.reader(f)

        for row in reader:
            if not isColumnHeader:
                patients.append(row[0])
            isColumnHeader = False

    for patient in patients:
        #print(patient)
        inputData.append(process_data(patient, img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT))

    #prediction!
    prediction = sess.run(output, feed_dict={x: inputData})
    print(prediction)
    PrintPrediction(patients, prediction, 'prediction.csv')

Model restored.
Initialized
ac4056071f3cc98489b9db3aebfe2b6a
ae2fdcd8daa3fede6ae23cc63a8d9a82
ae4e9d8aab8f8f5ae975bcca923f468d
ae61ec94b0b8de5439180f4776551e42
aec5a58fea38b77b964007aa6975c049
af1d0c2fcde369dd1b715460c2f704a2
b0599ad2f33276e7cd065eaa8dcec8a2
b17c07114dcf49ce71c8da4b43cf1192
b4d5b618fdf3a5a1bcfb325a3715e99e
b4db5b96c65a668a2e63f9a3ed36afe7
b53d997901eb880c41fbfbc82847204c
b6857d98b7b3dbe84f153617f4dfd14b
b82efe72526c59a96257208d95e54baf
b8793dbd40de88c0de0913abbaab0fe7
bbf7a3e138f9353414f2d51f0c363561
bdc2daa372a36f6f7c72abdc0b5639d1
bdfb2c23a8c1dca5ea8c1cc3d89efee9
be3e35bf8395366d235b8bcfc71a05ee
be9a2df5a16434e581c6a0625c290591
bf6a7a9ab4e18b18f43129c9e22fb448
c0c5a155e6e59588783c2964975e7e1e
c25876fb40d6f8dafd1ecb243193dd3f
c2ef34cc347bc224b5a123426009d027
c3a9046fbe2b0f0a4e43a669c321e472
c46c3962c10e287f1c1e3af0d309a128
c71d0db2086b7e2024ca9c11bd2ca504
c7bdb83b7ca6269fac16ab7cff930a2e
c87a713d17522698958de55c97654beb
c95f2aa23e6d6702f5b16a3b35f89cf0
cbb9bbd994c235b